# Example 1

In this notebook example 1.1 from the book **Mechanics of Materials** (10th Edition in SI units) by R.C. Hibbeler will be solved using the *python-mechanics* library. To do this exercise, we need to import the following items from this library:

In [1]:
from mechanics import Quantity
from mechanics.statics import (
    DistributedLoad2D,
    FixedEnd2D,
    Beam2D
)

In [2]:
Q_ = Quantity

It is asked to determine the resultant internal loadings acting on the cross section at C of the cantilevered beam shown in the image below.

![image](./images/example_1-1.png)

## Define the Loads Acting on the Beam

### Distributed Loading

To define the distributed loading, the `DistributedLoad2D` class will be used and certain points on the loading diagram are to be specified. For the given diagram, it suffices to specify two points. The first point is at the left end of the beam (point A), where we also locate the origin of our coordinate system (the x-coordinate of point A is at 0 m). The second point is at the right end of the beam (point B). The x-coordinate of point B is at 3.6 m. The load per unit length at point A is 300 N/m and vertically downward. The x-axis of our coordinate system coincides with the horizontal centerline of the beam and is pointing to the right. As the coordinate system must be right-handed, the y-axis, going through the origin at point A, runs vertically and is pointing upward. Because the direction of the distributed loading is opposite to the positive sense of the y-axis, we need to add a minus sign to the load at point A. At point B the load is zero.

In [3]:
q = DistributedLoad2D(
    points=[
        (Q_(0, 'm'), Q_(-300, 'N / m')),
        (Q_(3.6, 'm'), Q_(0, 'N / m'))
    ]
)

Internally, the distributed loading will be replaced by an equivalent single concentrated force going through the centroid (geometric center) of the area under the distributed loading diagram. 

In [4]:
Q = q.resultant()

print(Q)
print(Q.action_point)

<x: 0.000 N; y: -540.000 N; z: 0.000 N>
(<Quantity(1.2, 'meter')>, <Quantity(0, 'meter')>, <Quantity(0, 'meter')>)


### Fixed Wall Support

At point A, the beam is fixed in a wall. A fixed support may exert a horizontal as well as a vertical reaction force, and also a reaction couple moment (reaction torque). To define the fixed end of the beam on the left, it suffices to use the `FixedEnd2D` class. We need to give the support a name and specify its position with respect to the origin of the coordinate system that we have chosen.   

In [5]:
A = FixedEnd2D(
    name='A',
    position=(Q_(0, 'm'), Q_(0, 'm'))
)

## Define the Beam

Once the loads and the supports of a beam are defined, we can define the beam using the `Beam2D` class and add the loads to it. To define the beam, we also need to specify its length.

In [6]:
beam = Beam2D(
    length=Q_(3.6, 'm'), 
    loads=[q], 
    supports=[A]
)

### Reaction Forces and Torque at Point A

Now, we can get the reaction forces at the fixed end (point A) of the beam, by calling the method `get_reactions()` on our `Beam2D` instance.

In [7]:
forces, torques = beam.get_reactions()

for name, force in forces.items():
    print(f"{name}: {force}")

for name, torque in torques.items():
    print(f"{name}: {torque}")

A: <x: -0.000 N; y: 540.000 N; z: 0.000 N>
A: <x: 0.000 N·m; y: 0.000 N·m; z: 648.000 N·m>


At point A the fixed wall support exerts a vertical reaction force on the beam in the positive y-direction, i.e. upward. It also exerts a positive reaction torque along the z-axis. As the positive x-axis of our right-handed coordinate system points to the right, and the positive y-axis points upward, the positive direction of the z-axis is determined by the right-hand rule: when the fingers of our right hand are curled from the positive x-axis toward the positive y-axis (i.e. in the counter-clockwise sense), our right thumb will indicate the positive z-axis direction. As the reaction torque exerted by the fixed wall support has a positive sign, it operates in a counter-clockwise sense about point A of the beam.

### Resultant Internal Loadings at Point C

To determine the resultant internal loadings at point C, we make an imaginary section or straight cut through the beam at point C. Then, we can determine the internal forces and moments in the cross-section of the beam at point C such that the left or right part of the beam remains in static equilibrium with the external forces that are acting on that part of the beam. 

To make the cut and determine the internal forces and/or moments, we will call the method `cut(...)` on our `Beam2D` instance. We need to indicate the position along the beam where we want to make the cut (only the x-coordinate of this position is sufficient, as the centerline of the beam lies on the x-axis of our coordinate system) and we also need to specify which part of the beam we want to consider: either the left part (this is the default), or the right part.

> It can be noticed here that, depending on the specific situation, it might be unnecessary to determine first the external reaction forces and/or moments which are acting on the beam. In this example, if we would take the right part of the beam, we actually don't need to know the reaction forces/torques at point A to be able to find the resultant internal loadings at point C.

In [8]:
internal_load_C = beam.cut(x=Q_(1.2, 'm'), side='left')

print(
    internal_load_C['F_x'],
    internal_load_C['F_y'],
    internal_load_C['M_z']
)

0.0 newton -240.0 newton -192.0 meter * newton


Method `cut(...)` returns a dictionary with three keys: the axial force component `F_x` that pushes or pulls on the cross-section, the transversal force component `F_y` that exerts a shear force in the cross-section, and the moment `M_z` that exerts a bending moment in the cross-section. In this example the axial component is zero, as there are no external axial loads acting on the beam. In the left-sided cross-section at point C of the beam a resultant shear force is present acting downward and also a negative, i.e. clockwise, bending moment, which pushes on the lower part of the left-sided cross-section and pulls on its upper part.

By considering the left part of the beam, it is necessary that the reaction forces/torques at point A are determined first, before the method `cut(...)` is called. Should we have decided to consider the right part of the beam, only the external distributed loading would have been needed to determine the internal loading in the right-sided cross-section at point C of the beam.

If we look at the right-sided cross-section in point C of the beam, we get:

In [9]:
internal_load_C = beam.cut(x=Q_(1.2, 'm'), side='right')

print(
    internal_load_C['F_x'],
    internal_load_C['F_y'],
    internal_load_C['M_z']
)

-0.0 newton 240.0 newton 192.0 meter * newton


We notice that the magnitudes of the internal shear force and the internal bending moment in the right-sided cross-section and in the left-sided cross-section are the same, but they have an opposite sign. This makes sense, as the internal forces and moments on both sides of the cross-section must be in static equilibrium.